# 9.3. Statistical Coding - 10 points

## (3pt) Answer these question:

### 1) Which of these codes cannot be Huffman codes for any probability assignment and why?

- a) {0, 10, 11}
- b) {00, 01, 10, 110}
- c) {01, 10}

### 2) Classes of codes. Consider the code {0, 01}.
- a) Is it prefix code?
- b) Is it uniquely decodable?
- c) It it nonsingular?

### 3) Optimal word lengths.
- a) Can l=(1, 2, 2) be the word lengths of a binary Huffman code?
- a) Can l=(2, 2, 3, 3) be the word lengths of a binary Huffman code?

## (7pt) Programming Assignment
- Implement Huffman encoding - build a Huffman tree and encode the file.
- Use the same tree for decoding.
- Use the same files as in Exercise 1
- Compare the resulting compression to the entropy.
- Do not generate bits, string of 0s and 1s are sufficient.

In [1]:
import os
import sys

def adding_module_path():
    module_path = os.path.abspath(os.path.sep.join([".."]*2))

    if module_path not in sys.path:
        sys.path.append(module_path)

adding_module_path()

In [85]:
from src.load_data import get_dataset
from src.load_data import DataSets
from src.get_probs import get_sorted_probs_as_df
import numpy as np
import pandas as pd

In [3]:
data_dna = get_dataset(DataSets.dna)

Loading C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB


In [4]:
sorted_df = get_sorted_probs_as_df(data_dna)

Calculating freqs
End of calculating 8.338574409484863
Calculating probs
End of calculating 0.0009951591491699219


In [6]:
sorted_df

,0
A,2.899787e-01
T,2.897280e-01
C,2.102979e-01
G,2.099745e-01
N,8.907318e-06
\n,6.408691e-06
R,1.296997e-06
Y,1.087189e-06
K,8.773804e-07
W,8.583069e-07


In [53]:
class Node:
    def __init__(self, parent=None, left=None, right=None, prob=None, label=None):
        self.left = left
        self.right = right
        self.parent = parent
        self.prob = prob
        self.label = label

    def __str__(self):
        return f"{self.label} -- {self.prob}"

In [80]:
def process_node(node, alphabet, codeword = ""):
    if len(node.label) == 1:
        alphabet[node.label] = codeword

    if node.right is not None:
        process_node(node.right, alphabet, codeword + "1")

    if node.left is not None:
        process_node(node.left, alphabet, codeword + "0")

In [83]:
def build_encoding_table(tree):
    table = {}
    process_node(tree.root, table, "")
    return table

In [136]:
class HuffmanTree:
    def __init__(self):
        pass

    def get_char_encodings(self, alphabet):
        res = {}

        self.root


    def print_node(self, node, level):
        tabs = "\t" * level
        print(f'{tabs} {node}')

        if node.left is not None:
            self.print_node(node.left, level+1)

        if node.right is not None:
            self.print_node(node.right, level+1)


    def print_tree(self):
        self.print_node(self.root, 0)


    def get_left_right(self, last_two):
        left_label = last_two[0][0]
        left_prob = last_two[0][1]

        right_label = last_two[1][0]
        right_prob = last_two[1][1]

        left, right = list(sorted([(left_label, left_prob), (right_label, right_prob)], key=lambda item: len(item[0])))

        left_label = left[0]
        left_prob = left[1]

        right_label = right[0]
        right_prob = right[1]

        return left_label, left_prob, right_label, right_prob

    def get_nodes(self, existing_nodes, last_two):
        left, right, parent = None, None, None

        left_label = last_two[0][0]
        left_prob = last_two[0][1]

        right_label = last_two[1][0]
        right_prob = last_two[1][0]


        left_label, left_prob, right_label, right_prob = self.get_left_right(last_two)

        if left_label in existing_nodes:
            left = existing_nodes[left_label]
        else:
            left = Node(None, None, None, left_prob, left_label)
            existing_nodes[left_label] = left


        if right_label in existing_nodes:
            right = existing_nodes[right_label]
        else:
            right = Node(None, None, None, right_prob, right_label)
            existing_nodes[right_label] = right

        new_node_label = "".join(list(sorted([i[0] for i in last_two])))
        new_node_prob = np.sum([i[1] for i in last_two])

        parent = Node(None, left, right, new_node_prob, new_node_label)
        existing_nodes[new_node_label] = parent
        left.parent = parent
        right.parent = parent

        return left, right, parent

    def build_tree(self, data):
        #sort by probability
        sorted_df = get_sorted_probs_as_df(data_dna)

        dic = sorted_df.to_dict()[0]
        existing_nodes = {}
        parent = None

        while len(list(dic.keys())) >= 2:
            sorted_dic = sorted(dic.items(), key=lambda item: item[1], reverse=False)
            last_two = sorted_dic[0:2]
            
            left, right, parent = self.get_nodes(existing_nodes, last_two)


            new_dic = sorted_dic.copy()

            del new_dic[0:2]


            new_dic = dict(new_dic) 
            new_dic[parent.label] = parent.prob

            dic = new_dic
        
        self.root = parent

    def decode_binary(self, binary_text):

        pass
        #binary text

        #go over tree

        #if leaf return char


    def encode_char(self, char):
        #get builded tree

        #go over tree

        #if leaf return binary
        pass

    
    def encode(self, data):
        #iterate over char and create long string
        table = build_encoding_table(self)
        encoded = ""
        for c in data:
            encoded += table[c]
        return encoded

    def decode(self, data):
        decoded = ""
        #iterate over stream and get all decoded values
        ptr = self.root

        for bit in data:
            if 1 == len(ptr.label):
                decoded += ptr.label
                ptr = self.root

            if bit == '1':
                ptr = ptr.right

            if bit == '0':
                ptr = ptr.left
        
        decoded += ptr.label
        
        return decoded

In [137]:
tree = HuffmanTree()

In [138]:
tree.build_tree(data_dna)

Calculating freqs
End of calculating 8.775429010391235
Calculating probs
End of calculating 0.0


In [139]:
print(tree.root)


BDHVMSKWRYNGCAT -- 1.0


In [140]:
test = data_dna[0:10]
test

'GATCAATGAG'

In [141]:
encoded_data = tree.encode(test)
encoded_data

'11001001001010011001110'

In [142]:
encoded_data

'11001001001010011001110'

In [143]:
decoded_data = tree.decode(encoded_data)
decoded_data

'GATCAATGAG'

In [84]:
table = build_encoding_table(tree)

In [92]:
print(tree.root.right)

AT -- 0.5797067642211914


In [93]:
print(tree.root.left)


BDHVMSKWRYNGC -- 0.4202932357788086


In [88]:
pd.DataFrame.from_dict(table, orient='index')

,0
A,11
T,10
C,01
G,001
\n,00011
K,00010111
W,00010110
M,000101011
B,000101010111
D,000101010110
